### Objetivo: Consumir API

Acessando a url https://dadosabertos.bcb.gov.br/dataset/estatisticas-meios-pagamentos vocês terão acesso aos dados fornecidos pelo banco central sobre meios de pagamentos.
Vocês podem consultar a API para obter a url para requisição dos dados.


Determine a url fornecida:

In [18]:
url = "https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/EstatisticasTransacoesPix(Database=@Database)?@Database='202301'&$top=100&$format=json&$select=AnoMes,PAG_PFPJ,REC_PFPJ,PAG_REGIAO,REC_REGIAO,PAG_IDADE,REC_IDADE,FORMAINICIACAO,NATUREZA,FINALIDADE,VALOR,QUANTIDADE"

Importar bibliotecas

In [ ]:
import pandas as pd  #possibilita manipulação e análise de dados
import requests  #possibilita integração com serviços web, solicitações HTTP consumir dados de APIs
import json  #manipular arquivo json
from pymongo import MongoClient
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

Criar uma função para obter os dados:

In [20]:

def requisicao_api(link):
  resposta = requests.get(link)

  if resposta.status_code == 200:      #sucesso na requisição
    dados = resposta.json()
    print('Status Code:', resposta.status_code)
    with open('resultado.json', 'w', encoding='utf-8') as arquivo:   #salvar arquivo json
      json.dump(dados, arquivo, ensure_ascii=False, indent=4)

  else:
    print('Status Code:', resposta.status_code)


Chamar função passando o link gerado pela interface da API

In [21]:
requisicao_api(url)

Status Code: 200


Convertendo em DataFrame

In [22]:
jsondata = pd.read_json('resultado.json')

In [23]:
jsondata

,@odata.context,value
0,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202405, 'PAG_PFPJ': 'PF', 'REC_PFPJ..."
1,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202303, 'PAG_PFPJ': 'PF', 'REC_PFPJ..."
2,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202301, 'PAG_PFPJ': 'PJ', 'REC_PFPJ..."
3,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202401, 'PAG_PFPJ': 'PJ', 'REC_PFPJ..."
4,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202408, 'PAG_PFPJ': 'PF', 'REC_PFPJ..."
...,...,...
95,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202408, 'PAG_PFPJ': 'PJ', 'REC_PFPJ..."
96,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202406, 'PAG_PFPJ': 'PF', 'REC_PFPJ..."
97,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202412, 'PAG_PFPJ': 'PF', 'REC_PFPJ..."
98,https://was-p.bcnet.bcb.gov.br/olinda/servico/...,"{'AnoMes': 202311, 'PAG_PFPJ': 'PF', 'REC_PFPJ..."


In [24]:
data = jsondata['value']

In [25]:
dados_df = pd.json_normalize(data)

Exibindo os primeiros valores do Dataframe

In [26]:
dados_df.head()

,AnoMes,PAG_PFPJ,REC_PFPJ,PAG_REGIAO,REC_REGIAO,PAG_IDADE,REC_IDADE,FORMAINICIACAO,NATUREZA,FINALIDADE,VALOR,QUANTIDADE
0,202405,PF,PJ,Nao informado,SUDESTE,entre 30 e 39 anos,Nao se aplica,DICT,P2G,Pix,3384.22,8
1,202303,PF,PJ,NORTE,NORTE,mais de 60 anos,Nao se aplica,QRDN,P2B,Pix Troco,7525.59,16
2,202301,PJ,PF,SUDESTE,Nao informado,Nao se aplica,entre 30 e 39 anos,Nao disponivel,B2P,Nao disponivel,255424.33,1194
3,202401,PJ,PJ,SUL,NORDESTE,Nao se aplica,Nao se aplica,MANU,B2G,Pix,668315.98,77
4,202408,PF,PF,NORDESTE,SUL,entre 50 e 59 anos,entre 20 e 29 anos,MANU,P2P,Pix,1226112.51,8636


In [27]:
dados_df.isnull().sum()

AnoMes            0
PAG_PFPJ          0
REC_PFPJ          0
PAG_REGIAO        0
REC_REGIAO        0
PAG_IDADE         0
REC_IDADE         0
FORMAINICIACAO    0
NATUREZA          0
FINALIDADE        0
VALOR             0
QUANTIDADE        0
dtype: int64

In [28]:
dados_df.dtypes

AnoMes              int64
PAG_PFPJ           object
REC_PFPJ           object
PAG_REGIAO         object
REC_REGIAO         object
PAG_IDADE          object
REC_IDADE          object
FORMAINICIACAO     object
NATUREZA           object
FINALIDADE         object
VALOR             float64
QUANTIDADE          int64
dtype: object

In [29]:
dados_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AnoMes          100 non-null    int64  
 1   PAG_PFPJ        100 non-null    object 
 2   REC_PFPJ        100 non-null    object 
 3   PAG_REGIAO      100 non-null    object 
 4   REC_REGIAO      100 non-null    object 
 5   PAG_IDADE       100 non-null    object 
 6   REC_IDADE       100 non-null    object 
 7   FORMAINICIACAO  100 non-null    object 
 8   NATUREZA        100 non-null    object 
 9   FINALIDADE      100 non-null    object 
 10  VALOR           100 non-null    float64
 11  QUANTIDADE      100 non-null    int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 9.5+ KB


In [30]:
dados_df = dados_df.rename(columns={"PAG_PFPJ": "NATUREZA_PAGADOR"})

In [31]:
dados_df = dados_df.rename(columns={"REC_PFPJ": "NATUREZA_RECEBEDOR"})

In [ ]:
mongo_uri = os.getenv("MONGO_URI")

client = MongoClient(mongo_uri)

In [33]:
db = client["Pix"]

colecao = db["Pix_Transacoes"]

In [34]:
colecao.insert_many(dados_df.to_dict("records"))

InsertManyResult([ObjectId('68d5df754023d75558b00407'), ObjectId('68d5df754023d75558b00408'), ObjectId('68d5df754023d75558b00409'), ObjectId('68d5df754023d75558b0040a'), ObjectId('68d5df754023d75558b0040b'), ObjectId('68d5df754023d75558b0040c'), ObjectId('68d5df754023d75558b0040d'), ObjectId('68d5df754023d75558b0040e'), ObjectId('68d5df754023d75558b0040f'), ObjectId('68d5df754023d75558b00410'), ObjectId('68d5df754023d75558b00411'), ObjectId('68d5df754023d75558b00412'), ObjectId('68d5df754023d75558b00413'), ObjectId('68d5df754023d75558b00414'), ObjectId('68d5df754023d75558b00415'), ObjectId('68d5df754023d75558b00416'), ObjectId('68d5df754023d75558b00417'), ObjectId('68d5df754023d75558b00418'), ObjectId('68d5df754023d75558b00419'), ObjectId('68d5df754023d75558b0041a'), ObjectId('68d5df754023d75558b0041b'), ObjectId('68d5df754023d75558b0041c'), ObjectId('68d5df754023d75558b0041d'), ObjectId('68d5df754023d75558b0041e'), ObjectId('68d5df754023d75558b0041f'), ObjectId('68d5df754023d75558b004